In [7]:
import pickle
from util_fdp import *

In [8]:
mode='federated'
T_mode='rounds'  # samples / rounds
gene_process = 'homo' # 'homo /hete' / 'hete_d'


In [10]:
cvg_case2 = load_pickle(f'output/case_{mode}_{T_mode}_{gene_process}_cvg.pkl')
mae_case2 = load_pickle(f'output/case_{mode}_{T_mode}_{gene_process}_mae.pkl')

print(cvg_case2)
print(mae_case2)

{5000: {0: {0.25: {1: np.float64(0.954), 5: np.float64(0.974), 'log': np.float64(0.986)}, 'hetero': {1: np.float64(0.98), 5: np.float64(0.988), 'log': np.float64(0.996)}, 0.9: {1: np.float64(0.999), 5: np.float64(1.0), 'log': np.float64(1.0)}}, 1: {0.25: {1: np.float64(0.957), 5: np.float64(0.974), 'log': np.float64(0.991)}, 'hetero': {1: np.float64(0.99), 5: np.float64(0.99), 'log': np.float64(0.99)}, 0.9: {1: np.float64(1.0), 5: np.float64(1.0), 'log': np.float64(1.0)}}, 2: {0.25: {1: np.float64(0.956), 5: np.float64(0.975), 'log': np.float64(0.993)}, 'hetero': {1: np.float64(0.976), 5: np.float64(0.99), 'log': np.float64(0.995)}, 0.9: {1: np.float64(0.998), 5: np.float64(0.997), 'log': np.float64(1.0)}}}, 50000: {0: {0.25: {1: np.float64(0.956), 5: np.float64(0.975), 'log': np.float64(0.992)}, 'hetero': {1: np.float64(0.968), 5: np.float64(0.977), 'log': np.float64(0.995)}, 0.9: {1: np.float64(1.0), 5: np.float64(0.999), 'log': np.float64(0.998)}}, 1: {0.25: {1: np.float64(0.957), 5

In [11]:
# print(cvg_case2)
# print(mae_case2)

In [12]:
def generate_combined_table(data_ci, data_mae, caption="Results",
                            label="tab:combined"):
    # 自动获取所有n值并按数字顺序排序
    all_n = sorted(set(list(data_ci.keys()) + list(data_mae.keys())), key=int)
    latex_code = f""
    for n in all_n:
        # 添加n标题行
        latex_code += f"\\multicolumn{{6}}{{c}}{{\\textbf{{T = {n}}}}} \\\\\n"
        latex_code += "\\midrule\n"
        
        # 遍历所有tau和r组合
        for tau in sorted(data_ci[n].keys(), key=float):
            for r_val in data_ci[n][tau].keys():
                # 生成行内容
                row = f"{tau} & {r_val if isinstance(r_val, str) else r_val} "
                
                # 遍历所有E值
                for e in data_ci[n][tau][r_val].keys():
                    ci = data_ci[n][tau][r_val].get(e, "--")
                    mae = data_mae[n][tau][r_val].get(e, "--")
                    cell = f"& {ci:.3f}({mae:.4f}) " if isinstance(ci, float) and isinstance(mae, float) else "& -- "
                    row += cell
                
                latex_code += row + "\\\\\n"
            
            # 添加tau之间的分隔线
            latex_code += "\\hline\n"
    latex_code += """\\bottomrule
\\end{tabular}
\\end{table}"""
    return latex_code

In [13]:
latex_output = generate_combined_table(cvg_case2, mae_case2)
with open(f"output/tex_{mode}_{T_mode}_{gene_process}.tex", "w") as f:
    f.write(latex_output)

## For Global

In [2]:
mode='global'
gene_process = 'homo' # 'homo /hete' / 'hete_d'


In [3]:
cvg_case2 = load_pickle(f'output/case_{mode}_{gene_process}_cvg.pkl')
mae_case2 = load_pickle(f'output/case_{mode}_{gene_process}_mae.pkl')

print(cvg_case2)
print(mae_case2)

{10000: {0: {0.25: np.float64(0.979), 'hetero': np.float64(0.999), 0.9: np.float64(1.0)}, 1: {0.25: np.float64(0.975), 'hetero': np.float64(0.999), 0.9: np.float64(1.0)}, 2: {0.25: np.float64(0.986), 'hetero': np.float64(1.0), 0.9: np.float64(1.0)}}, 50000: {0: {0.25: np.float64(0.98), 'hetero': np.float64(1.0), 0.9: np.float64(1.0)}, 1: {0.25: np.float64(0.975), 'hetero': np.float64(0.999), 0.9: np.float64(1.0)}, 2: {0.25: np.float64(0.968), 'hetero': np.float64(0.983), 0.9: np.float64(0.927)}}}
{10000: {0: {0.25: np.float64(0.06594225688624544), 'hetero': np.float64(0.029231739484614962), 0.9: np.float64(0.014233008604130112)}, 1: {0.25: np.float64(0.06854195240171665), 'hetero': np.float64(0.030863929065240117), 0.9: np.float64(0.014750660006723844)}, 2: {0.25: np.float64(0.07171200421752542), 'hetero': np.float64(0.030461311054458994), 0.9: np.float64(0.015966506917189857)}}, 50000: {0: {0.25: np.float64(0.019650075538788732), 'hetero': np.float64(0.008053381392343103), 0.9: np.flo

In [4]:
def generate_combined_table(data_ci, data_mae,
                            caption="Results",
                            label="tab:combined"):
    r"""
    生成 LaTeX 表格，行格式：
        tau  &  r  &  ci(mae)  \\
    适用于 data_ci[n][tau][r]  以及  data_mae[n][tau][r]  这种三层嵌套结构
    """

    # ── 收集并排序所有 T ───────────────────────────────────────────
    all_n = sorted(set(list(data_ci.keys()) + list(data_mae.keys())),
                   key=int)

    latex_code = (
        "\\begin{table}[htbp]\n"
        "\\centering\n"
        f"\\caption{{{caption}}}\n"
        f"\\label{{{label}}}\n"
        "\\begin{tabular}{lll}\n"   # 3 列：tau, r, ci(mae)
        "\\toprule\n"
        "τ & r & CI(MAE)\\\\\n"
        "\\midrule\n"
    )

    for n in all_n:
        # ★ 3 列 → \multicolumn{3}
        latex_code += f"\\multicolumn{{3}}{{c}}{{\\textbf{{T = {n}}}}} \\\\\n"
        latex_code += "\\midrule\n"

        # 遍历 tau
        for tau in sorted(data_ci[n].keys(), key=float):
            # 遍历 r
            for r_val in data_ci[n][tau].keys():
                ci  = data_ci[n][tau].get(r_val, "--")
                mae = data_mae[n][tau].get(r_val, "--")

                cell = (
                    f"{ci:.3f}({mae:.4f})"
                    if isinstance(ci, float) and isinstance(mae, float)
                    else "--"
                )
                row = (
                    f"{tau} & "
                    f"{r_val if isinstance(r_val, str) else r_val} & "
                    f"{cell} \\\\"
                )
                latex_code += row + "\n"

            # tau 之间分隔线
            latex_code += "\\hline\n"

    latex_code += (
        "\\bottomrule\n"
        "\\end{tabular}\n"
        "\\end{table}"
    )
    return latex_code

In [5]:
latex_output = generate_combined_table(cvg_case2, mae_case2)
with open(f"output/tex_{mode}_{gene_process}.tex", "w") as f:
    f.write(latex_output)